In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="answer", description="回答内容"),
    ResponseSchema(name="confidence", description="置信度，0-1之间的浮点数")
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)
prompt = f"请按照以下格式返回数据：\n{parser.get_format_instructions()}"

prompt





In [7]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pydantic import ValidationError
from dotenv import  load_dotenv
load_dotenv()
import os



#  密钥（替换为你的密钥，或通过环境变量获取） 这里我用的阿里云的

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")
model = "qwq-32b"


# 定义期望的输出结构
response_schemas = [
    ResponseSchema(name="answer", description="回答内容，字符串类型"),
    ResponseSchema(name="confidence", description="置信度，0-1之间的浮点数")
]

# 创建输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 定义提示模板
prompt_template = """
请分析以下句子的情感，并以指定的 JSON 格式返回结果：
{format_instructions}

句子：{sentence}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["sentence"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# 初始化语言模型
llm = ChatOpenAI(temperature=0.2, max_tokens=200,api_key = api_key , base_url = base_url,model = "qwq-32b",streaming = True)

# 定义解析和重试逻辑
def get_stable_output(sentence, max_attempts=3):
    chain = prompt | llm | output_parser
    
    for attempt in range(max_attempts):
        try:
            # 调用模型并解析输出
            result = chain.invoke({"sentence": sentence})
            return result
        except Exception as e:
            print(f"尝试 {attempt + 1}/{max_attempts} 失败: {e}")
            if attempt == max_attempts - 1:
                # 最后一次尝试失败，返回默认值
                return {"answer": "无法分析", "confidence": 0.5}
    
    return {"answer": "未完成分析", "confidence": 0.0}  # 理论上不会到达这里

# 测试代码
def main():
    test_sentences = [
        "我今天感觉非常开心！",
        "这个世界真是让人失望。",
        "天气很好，适合出门散步。"
    ]

    for sentence in test_sentences:
        print(f"\n分析句子：{sentence}")
        result = get_stable_output(sentence)
        print(f"结果：{result}")

if __name__ == "__main__":
    main()


分析句子：我今天感觉非常开心！
结果：{'answer': '正面', 'confidence': '0.99'}

分析句子：这个世界真是让人失望。
结果：{'answer': '负面', 'confidence': '0.95'}

分析句子：天气很好，适合出门散步。
结果：{'answer': '积极', 'confidence': '0.95'}


In [8]:
from langchain.agents import initialize_agent, AgentType